<a href="https://colab.research.google.com/github/Chaotic-Legend/CMP-414-Codes/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Homework 2
import random
# ---------------- Agent Class ----------------
class Agent:
    def __init__(self, grid, target, start=None):
        self.grid = grid
        self.num_rows = len(grid)
        self.num_cols = len(grid[0])
        self.target = target

        if start is None:
            while True:
                r = random.randint(0, self.num_rows - 1)
                c = random.randint(0, self.num_cols - 1)
                if [r, c] != target and grid[r][c] != "X":
                    start = [r, c]
                    break

        self.position = start[:]               # Current Position [row, col]
        self.visited = [start[:]]              # List of visited positions
        self.path = [start[:]]                 # Path from start to current

    def perceive_environment(self):
        row, col = self.position
        up = "Wall" if row == 0 else ("Obstacle" if self.grid[row-1][col] == "X" else "Empty")
        down = "Wall" if row == self.num_rows-1 else ("Obstacle" if self.grid[row+1][col] == "X" else "Empty")
        left = "Wall" if col == 0 else ("Obstacle" if self.grid[row][col-1] == "X" else "Empty")
        right = "Wall" if col == self.num_cols-1 else ("Obstacle" if self.grid[row][col+1] == "X" else "Empty")
        return up, down, left, right

    def decide_action(self):
        up, down, left, right = self.perceive_environment()
        row, col = self.position
        if right == "Empty" and [row, col+1] not in self.visited:
            return "Right"
        elif down == "Empty" and [row+1, col] not in self.visited:
            return "Down"
        elif up == "Empty" and [row-1, col] not in self.visited:
            return "Up"
        elif left == "Empty" and [row, col-1] not in self.visited:
            return "Left"
        else:
            return "Backtrack"

    def take_action(self, action=None):
        if action is None:
            action = self.decide_action()

        row, col = self.position

        if action == "Right":
            col += 1
        elif action == "Down":
            row += 1
        elif action == "Up":
            row -= 1
        elif action == "Left":
            col -= 1
        elif action == "Backtrack":
            if len(self.path) > 1:
                self.path.pop()
                row, col = self.path[-1]
            else:
                row, col = self.position

        self.position = [row, col]

        if self.position not in self.visited:
            self.visited.append(self.position[:])

        if action != "Backtrack":
            self.path.append(self.position[:])

# ---------------- Environment Creation ----------------
def create_environment(num_rows, num_cols, num_obstacles):
    grid = [["_" for _ in range(num_cols)] for _ in range(num_rows)]

    while True:
        target_row = random.randint(0, num_rows - 1)
        target_col = random.randint(0, num_cols - 1)
        if grid[target_row][target_col] == "_":
            break
    target = [target_row, target_col]
    grid[target_row][target_col] = "T"

    count = 0
    while count < num_obstacles:
        r = random.randint(0, num_rows - 1)
        c = random.randint(0, num_cols - 1)
        if grid[r][c] == "_" and [r, c] != target:
            grid[r][c] = "X"
            count += 1

    return grid, target

# ---------------- Grid Display ----------------
def print_grid(grid, agent = None):
    display = [row.copy() for row in grid]

    if agent:
        # Mark visited path with "."
        for r, c in agent.path:
            if display[r][c] == "_":
                display[r][c] = "."

        # Always keep the starting position as "0"
        start_r, start_c = agent.path[0]
        display[start_r][start_c] = "0"

        # If the agent has reached the target, show "A" at the target cell.
        if agent.position == agent.target:
            r, c = agent.position
            display[r][c] = "A"

    for row in display:
        print(" ".join(row))
    print()

# ---------------- Simulation ----------------
# Create environment with arbitrary target.
grid, target_pos = create_environment(5, 7, 5)
agent = Agent(grid, target=target_pos)

print("Arbitrary Grid With Arbitrary Target:")
print_grid(grid, agent)

actions_taken = []
positions_taken = []

while agent.position != agent.target:
    action = agent.decide_action()
    agent.take_action(action)
    actions_taken.append(action)
    positions_taken.append(agent.position.copy())

print("All the steps to reach the target:")
print(f"Step 0 - Action: Start, Position: {agent.path[0]}")
for step, (action, pos) in enumerate(zip(actions_taken, positions_taken), start=1):
    print(f"Step {step} - Action: {action}, Position: {pos}")

print(f"\nThe agent has reached the target at position: {agent.position}")
print("\nVisited Cells:", agent.visited)
print("\nCorrect Path:", agent.path)
print("\nFinal Grid With Correct Path:")
print_grid(grid, agent)

Arbitrary Grid With Arbitrary Target:
X _ _ _ _ X _
_ X _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ T _ _ _
_ X _ _ _ X 0

All the steps to reach the target:
Step 0 - Action: Start, Position: [4, 6]
Step 1 - Action: Up, Position: [3, 6]
Step 2 - Action: Up, Position: [2, 6]
Step 3 - Action: Up, Position: [1, 6]
Step 4 - Action: Up, Position: [0, 6]
Step 5 - Action: Backtrack, Position: [1, 6]
Step 6 - Action: Left, Position: [1, 5]
Step 7 - Action: Down, Position: [2, 5]
Step 8 - Action: Down, Position: [3, 5]
Step 9 - Action: Left, Position: [3, 4]
Step 10 - Action: Down, Position: [4, 4]
Step 11 - Action: Left, Position: [4, 3]
Step 12 - Action: Up, Position: [3, 3]

The agent has reached the target at position: [3, 3]

Visited Cells: [[4, 6], [3, 6], [2, 6], [1, 6], [0, 6], [1, 5], [2, 5], [3, 5], [3, 4], [4, 4], [4, 3], [3, 3]]

Correct Path: [[4, 6], [3, 6], [2, 6], [1, 6], [1, 5], [2, 5], [3, 5], [3, 4], [4, 4], [4, 3], [3, 3]]

Final Grid With Correct Path:
X _ _ _ _ X _
_ X _ _ _ . .
_ _ _ _